## Exercise

The company `SparkyCO` have noticed more than usual servers connectivity disruptions. You were given a task to investigate that and monitor all requests processed by the severs.

You need to process sever access logs, monitor incoming traffic and detect any anomaly in 1 hour windows.
The logs entries are qritten in format known as *Common Log Format* (CLF), standard format that can be produced by many different web servers and read by many log analysis programs.

Example log entry:
> 127.0.0.1 - - [26/Apr/2024:07:20:39 +0000] "GET /lists HTTP/1.0" 200 2503

Provided information from that entry: 
1. IP address of the client (remote host) that made request to the server
2. First "hypen" indicates that requested piece of information is not available
3. User if of the person reqesting. If the requested document does not need authentication, this part will also be just a "hypen"
4. Time that the request was received
5. The request line from the client: <br>
a. method usef ny client <br>
b. requested endpoint <br>
c. used protocol
6. Status code
7. Size of the object returned to the client



### Task description

1. Read incoming stream data using `text` output
2. Parse incoming log (use provided regex expressions or any other method)
3. Cast `timestamp` column to appropriate type
4. Set a watermatk for `2 hours`
5. Group by `host` with `1 hour` tumbling window based on `timestamp` and add `30 minutes` offset
6. Count how many times specific `host` tried to reach a server and alias the resulting column as `requests`
4. Filter "localhost" endpoints 
6. add boolean column "possible_threat", if `requests` is over 1_000
7. Write output to a file output

In [ ]:
# regex patterns to parse logs

host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
time_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}:\d{4})]'
general_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
status_pattern = r'\s(\d{3})\s'
content_size_pattern = r'\s(\d+)$'

## Setup


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

In [ ]:
spark = SparkSession.builder.appName("Log Analysis").getOrCreate()

### Generate data
For out practice purposes, we will generate mockup Apache Access logs.
>1. Open terminal and change directory to `data` folder
>2. In terminal, run command `python log_generator apache_access_schema.yaml`, add flag `-t` to truncate previously created logs
>3. Based on provide .yaml file, new `log.txt.{id}` file should appear every 5s with 50 lines of random logs in `log/apache_access` directory

**Remeber to exit from log generation script and close query connection at the end**

## Practice

1. Create DataFrame representing the stream of raw log data lines ariving to `/log/apache_access` directory

In [ ]:
access_lines = spark.readStream.text("../data/log/apache_access")

2. Parse log data to DataFrame

In [ ]:


# logsDF = access_lines.select(regexp_extract('value', hostExp, 1).alias('host'),
#                         regexp_extract('value', timeExp, 1).alias('timestamp'),
#                         regexp_extract('value', generalExp, 1).alias('method'),
#                         regexp_extract('value', generalExp, 2).alias('endpoint'),
#                         regexp_extract('value', generalExp, 3).alias('protocol'),
#                         regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
#                         regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

3. Count every access status code

In [ ]:
statusCountsDF=logsDF.groupBy("status").count()

4. Kick off our streaming query, dumping results to the console

In [ ]:
query = statusCountsDF.writeStream.outputMode("complete").format("console").queryName("status_counts").start()

5. Classroom cleanup
IMPORTANT: Kill running log_generator script in terminal

In [ ]:
query.stop()
spark.stop()